# Use this script on local machine to run demo


1.   Uses MTCNN for face detection
2.   Uses HSEmotion 'enet_b0_best_afew' EfficientNet model trained on AFEW dataset.

- Author: Ankita Jaswal

In [ ]:
import os

# dependencies
os.system('pip install hsemotion')
os.system('pip install mtcnn')

import cv2
import numpy as np
from mtcnn import MTCNN
from hsemotion.facial_emotions import HSEmotionRecognizer

# Initialize the HSEmotion recognizer
model_name = 'enet_b0_8_best_afew'
fer = HSEmotionRecognizer(model_name=model_name, device='cpu')  # 'cpu' or 'cuda'

# Initialize the face detector
detector = MTCNN()

# Start the webcam feed
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the captured frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the image
    detections = detector.detect_faces(frame_rgb)
    for detection in detections:
        x, y, width, height = detection['box']
        face_img = frame_rgb[y:y+height, x:x+width]

        # Predict emotion
        emotion_pred, scores = fer.predict_emotions(face_img, logits=True)
        maxindex = np.argmax(scores)
        emotion_label = fer.idx_to_class[maxindex]

        # Draw the detected face and write the emotion
        cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Check if window closed
    if cv2.getWindowProperty('Video', cv2.WND_PROP_VISIBLE) < 1:
        break

    # Exit upon pressing q key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()

# Now take some screenshots and plot the results

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# Directory of results
image_directory = 'path_to_saved_images_directory'

# Create a figure
plt.figure(figsize=(10, 10))

# Iterate over the files in the directory
for i, filename in enumerate(os.listdir(image_directory)):
    if filename.endswith(".jpg") or filename.endswith(".png", ".jpg", ".jpeg"):  # Add other file types if needed
        image_path = os.path.join(image_directory, filename)
        img = mpimg.imread(image_path)
        plt.subplot(5, 5, i+1)  # Adjust the grid size as needed
        plt.imshow(img)
        plt.axis('off')

plt.show()
